# ПРОЕКТ ПО SQL: АНАЛИЗ БАЗЫ ДАННЫХ КРУПНОГО СЕРВИСА ДЛЯ ЧТЕНИЯ КНИГ.
<a id = "start"></a>

__Описание проекта:__ Получена база данных сервиса для чтения книг по подписке, необходимо исследовать ее для дальнейшего развития компании во времена эпидемии коронавируса.

__Цель проекта:__ Проанализировать базу данных сервиса для чтения книг по подписке.

__Ход исследования:__

1. [Обзор данных.](#Обзор-данных)
2. [Рассчет количества книг вышедших после 1 января 2000 года.](#Рассчет-количества-книг-вышедших-после-1-января-2000-года.)
3. [Для каждой книги посчитать количество обзоров и среднюю оценку.](#Расчет-количества-обзоров-и-средней-оценки-для-каждой-книги)
4. [Определение издательства, которое выпустило наибольшее число книг толще 50 страниц.](#Определение-издательства,-которое-выпустило-наибольшее-число-книг-толще-50-страниц.)
5. [Определение автора с самой высокой средней оценкой книг.](#Определение-автора-с-самой-высокой-средней-оценкой-книг.)
6. [Расчет среднего количества обзоров от пользователей, которые поставили больше 48 оценок.](#Расчет-среднего-количества-обзоров-от-пользователей,-которые-поставили-больше-48-оценок.)
7. [Итоговый вывод.](#Итоговый-вывод-по-проекту.)



Вернуться в [начало проекта](#start)

## Обзор данных

Импортируем библиотеки для работы с SQL запросами. 

In [1]:
# импортируем библиотеки
import pandas as pd
import sqlalchemy as sa
from sqlalchemy import text, create_engine

Установим параметры для подключения к базе данных и сохраним коннектор.

In [2]:
# устанавливаем параметры
db_config = {
    'user': 'p', # имя пользователя
    'pwd': 'p', # пароль
    'host': 'p',
    'port': 7777, # порт подключения
    'db': 'data'
} # название базы данных

connection_string = 'postgresql://{u}:{p}@{h}:{p}/{d}'.format(
    **db_config
)

# сохраняем коннектор
engine = sa.create_engine(connection_string, connect_args={'sslmode':'require'})


Зададим функцию для выполнения SQL-запроса

In [3]:
# чтобы выполнить SQL-запрос, пишем функцию с использованием Pandas
def get_sql_data(query:str, engine:sa.engine.base.Engine=engine) -> pd.DataFrame:
    '''Открываем соединение, получаем данные из sql, закрываем соединение'''
    with engine.connect() as con:
        return pd.read_sql(sql=sa.text(query), con = con)

Исследуем таблицы - выведем первые строки и посчитаем количество строк в каждой таблице

In [4]:
# формируем запрос и выводим данные
query_books = '''
        SELECT * 
        FROM books 
        LIMIT 5
        '''
get_sql_data(query_books)

,book_id,author_id,title,num_pages,publication_date,publisher_id
0,1,546,'Salem's Lot,594,2005-11-01,93
1,2,465,1 000 Places to See Before You Die,992,2003-05-22,336
2,3,407,13 Little Blue Envelopes (Little Blue Envelope...,322,2010-12-21,135
3,4,82,1491: New Revelations of the Americas Before C...,541,2006-10-10,309
4,5,125,1776,386,2006-07-04,268


In [5]:
# выведем количество строк в таблице books
query_count_books = '''
        SELECT 'Количество строк в таблице "books":', 
                COUNT(*) 
        FROM books 
        LIMIT 5
        '''
get_sql_data(query_count_books)

,?column?,count
0,"Количество строк в таблице ""books"":",1000


In [6]:
# формируем запрос и выводим данные
query_authors = '''
        SELECT * 
        FROM authors 
        LIMIT 5
        '''
get_sql_data(query_authors)

,author_id,author
0,1,A.S. Byatt
1,2,Aesop/Laura Harris/Laura Gibbs
2,3,Agatha Christie
3,4,Alan Brennert
4,5,Alan Moore/David Lloyd


In [7]:
# выведем количество строк в таблице authors
query_count_authors = '''
        SELECT 'Количество строк в таблице "authors":', 
                COUNT(*) 
        FROM authors 
        LIMIT 5
        '''
get_sql_data(query_count_authors)

,?column?,count
0,"Количество строк в таблице ""authors"":",636


In [8]:
# формируем запрос и выводим данные
query_publishers = '''
        SELECT * 
        FROM publishers 
        LIMIT 5
        '''
get_sql_data(query_publishers)

,publisher_id,publisher
0,1,Ace
1,2,Ace Book
2,3,Ace Books
3,4,Ace Hardcover
4,5,Addison Wesley Publishing Company


In [9]:
# выведем количество строк в таблице publishers
query_count_publishers = '''
        SELECT 'Количество строк в таблице "publishers":', 
                COUNT(*) 
        FROM publishers 
        LIMIT 5
        '''
get_sql_data(query_count_publishers)

,?column?,count
0,"Количество строк в таблице ""publishers"":",340


In [10]:
# формируем запрос и выводим данные
query_ratings = '''
        SELECT * 
        FROM ratings 
        LIMIT 5
        '''
get_sql_data(query_ratings)

,rating_id,book_id,username,rating
0,1,1,ryanfranco,4
1,2,1,grantpatricia,2
2,3,1,brandtandrea,5
3,4,2,lorichen,3
4,5,2,mariokeller,2


In [11]:
# выведем количество строк в таблице ratings
query_count_ratings = '''
        SELECT 'Количество строк в таблице "ratings":', 
               COUNT(*) 
        FROM ratings 
        LIMIT 5
        '''
get_sql_data(query_count_ratings)

,?column?,count
0,"Количество строк в таблице ""ratings"":",6456


In [12]:
# формируем запрос и выводим данные
query_reviews = '''
        SELECT * 
        FROM reviews 
        LIMIT 5
        '''
get_sql_data(query_reviews)

,review_id,book_id,username,text
0,1,1,brandtandrea,Mention society tell send professor analysis. ...
1,2,1,ryanfranco,Foot glass pretty audience hit themselves. Amo...
2,3,2,lorichen,Listen treat keep worry. Miss husband tax but ...
3,4,3,johnsonamanda,Finally month interesting blue could nature cu...
4,5,3,scotttamara,Nation purpose heavy give wait song will. List...


In [13]:
# выведем количество строк в таблице reviews
query_count_reviews = '''
        SELECT 'Количество строк в таблице "reviews":', 
                COUNT(*) 
        FROM reviews 
        LIMIT 5
        '''
get_sql_data(query_count_reviews)

,?column?,count
0,"Количество строк в таблице ""reviews"":",2793


__Вывод:__
Данные содержатся в 5 таблицах:
- В таблице `books` 1000 строк и 6 колонок.
- В таблице `authors` 636 строк и 2 колонки.
- В таблице `publishers` 340 строк и 2 колонки.
- В таблице `ratings` 6456 строк и 4 колонки.
- В таблице `reviews` 2793 строки и 4 колонки.

Вернуться в [начало проекта](#start)

## Рассчет количества книг вышедших после 1 января 2000 года.

In [14]:
# формируем запрос и выводим данные
query_count_2000 = '''
        SELECT 'Количество книг вышедших после 1 января 2000 года:',
                COUNT(book_id) 
        FROM books 
        WHERE EXTRACT(YEAR FROM publication_date) >= 2000;
        '''
get_sql_data(query_count_2000)

,?column?,count
0,Количество книг вышедших после 1 января 2000 г...,821


__Согласно данным зарегистрированым в БД после 1 января 2000 года было выпущено книг 821.__

Вернуться в [начало проекта](#start)

## Расчет количества обзоров и средней оценки для каждой книги

In [15]:
# формируем запрос и выводим данные
query_avg_rating = '''
        SELECT bk.book_id,
               bk.title,
               COUNT(DISTINCT rev.review_id) AS count_review,
               ROUND(AVG(rat.rating)) AS avg_rating
        FROM books AS bk
        JOIN ratings AS rat ON bk.book_id = rat.book_id
        LEFT JOIN reviews AS rev ON bk.book_id = rev.book_id
        
        GROUP BY bk.book_id
        ORDER BY count_review DESC,
                 avg_rating DESC;
        '''
get_sql_data(query_avg_rating)

,book_id,title,count_review,avg_rating
0,948,Twilight (Twilight #1),7,4.0
1,854,The Road,6,4.0
2,627,The Alchemist,6,4.0
3,733,The Giver (The Giver #1),6,4.0
4,673,The Catcher in the Rye,6,4.0
...,...,...,...,...
995,191,Disney's Beauty and the Beast (A Little Golden...,0,4.0
996,83,Anne Rice's The Vampire Lestat: A Graphic Novel,0,4.0
997,221,Essential Tales and Poems,0,4.0
998,387,Leonardo's Notebooks,0,4.0


__Больше всего обзоров у книги `Twilight` - 7 обзоров, при этом средний рейтинг у этой книги 4.__

Вернуться в [начало проекта](#start)

## Определение издательства, которое выпустило наибольшее число книг толще 50 страниц.

In [16]:
# формируем запрос и выводим данные
query_top_publisher = '''
        SELECT pub.publisher_id,
               pub.publisher,
               COUNT(b.book_id) AS count_books
        FROM publishers AS pub
        JOIN books AS b ON pub.publisher_id = b.publisher_id
        WHERE b.num_pages > 50
        GROUP BY pub.publisher_id
        ORDER BY count_books DESC
        LIMIT 5;
        '''
get_sql_data(query_top_publisher)

,publisher_id,publisher,count_books
0,212,Penguin Books,42
1,309,Vintage,31
2,116,Grand Central Publishing,25
3,217,Penguin Classics,24
4,35,Bantam,19


__Издательтво `Penguin Books` выпустило наибольшее число книг (42 книги) толще 50 страниц.__

Вернуться в [начало проекта](#start)

## Определение автора с самой высокой средней оценкой книг.
Учитываем только книги с 50 и более оценками.

In [17]:
# формируем запрос и выводим данные
query_top_author = '''
        SELECT au.author_id,
               au.author,
               ROUND(AVG(rat.rating), 3) AS avg_rating
        FROM authors AS au
        JOIN books AS b ON au.author_id = b.author_id
        JOIN ratings AS rat ON b.book_id = rat.book_id
        WHERE b.book_id IN (SELECT book_id
                          FROM  ratings
                          GROUP BY book_id 
                          HAVING COUNT(rating_id) > 50)
        GROUP BY au.author_id
        ORDER BY avg_rating DESC
        LIMIT 5;
        '''
get_sql_data(query_top_author)

,author_id,author,avg_rating
0,236,J.K. Rowling/Mary GrandPré,4.287
1,402,Markus Zusak/Cao Xuân Việt Khương,4.264
2,240,J.R.R. Tolkien,4.247
3,376,Louisa May Alcott,4.192
4,498,Rick Riordan,4.081


__Автор с самой высокой средней оценков книг - J.K. Rowling/Mary GrandPré, средний рейтинг ее книг составил 4.287__

Вернуться в [начало проекта](#start)

## Расчет среднего количества обзоров от пользователей, которые поставили больше 48 оценок.

In [18]:
#  формируем запрос и выводим данные
query_avg_review = '''
        SELECT 'Среднее количество обзоров от пользователей:',
                AVG(rev.review_count)
        FROM (SELECT username,
               COUNT(review_id) AS review_count
        FROM reviews
        WHERE username IN (SELECT username
                           FROM ratings 
                           GROUP BY username
                           HAVING COUNT(rating_id) > 48)
        GROUP BY username) AS rev;
        '''
get_sql_data(query_avg_review)

,?column?,avg
0,Среднее количество обзоров от пользователей:,24.0


__В среднем количество обзоров, от пользователей поставивших больше 48 оценок, равняется 24, что состовляет 50% от минимального количество поставленных оценок среди таких пользователей. Можно предположить, что пользователи ставят оценкку в два раза чаще чем пишут отзыв.__

Вернуться в [начало проекта](#start)

## Итоговый вывод по проекту.

В ходе исследования базы данных сервиса для чтения книг выявлено:

- Таблица `books` содержит информацию о 1 000 книг.
- Таблица `authors` содержит информацию о 636 авторах.
- Таблица `publishers` содержит информацию о 340 издательствах.
- Таблица `ratings` содержит информацию о 6 456 пользовательских оценках книг.
- Таблица `reviews` содержит информацию о 2 793 пользовательских обзорах на книги.
- После 1 января 2000 года было выпущено 821 книга.
- Больше всего обзоров у книги `Twilight` - 7 обзоров, при этом средний рейтинг у этой книги 4.
- Издательтво `Penguin Books` выпустило наибольшее число книг (42 книги) толще 50 страниц.
- Автор с самой высокой средней оценков книг - `J.K. Rowling/Mary GrandPré`, средний рейтинг ее книг составил 4.287
- В среднем количество обзоров, от пользователей поставивших больше 48 оценок, равняется 24, что состовляет 50% от минимального количество поставленных оценок среди таких пользователей. Можно предположить, что пользователи ставят оценкку в два раза чаще чем пишут отзыв.

__Рекомендации:__
1. Мотивировать пользователей больше писать обзоров.
2. Выводить в рекомендованное книги авторов с наивысшим рейтингом, таких как  `J.K. Rowling/Mary GrandPré`, `Markus Zusak/Cao Xuân Việt Khương`, `J.R.R. Tolkien`.
3. Большая часть книг в сервисе современные, стоит расширить базу и книгами выпущенными в прошллом столетии.

Вернуться в [начало проекта](#start)